# Composite Selectors

Combining selectors is one of the most powerful features of `soupsavvy`. By allowing you to mix and match different selectors, `soupsavvy` offers a highly flexible and customizable way to refine your search criteria. In this demo, we'll explore **Higher Order Selectors** and demonstrate how they can be leveraged to perform complex searches effectively.

### Higher Order Selectors

Higher Order Selectors enable you to combine multiple selectors into a single, composite selector, enhancing your ability to target specific elements. Whether you're matching tags, attributes, or text content, these selectors can be used together to create more advanced search logic.

## Combinators

We'll begin with **Combinators**, which are inspired by the CSS concept of combinators. These allow you to pass multiple selectors and perform a search on a `Tag` object, mimicking the logic of CSS combinators but with the added power and flexibility of `soupsavvy`.

For more information on CSS combinators, you can refer to this [Mozilla guide](https://developer.mozilla.org/en-US/docs/Web/CSS/Child_combinator).

While `BeautifulSoup` provides methods like `select` and `select_one` to find tags using CSS selectors, these are often limited by the constraints of vanilla CSS. `soupsavvy` goes beyond these limitations, offering more complex conditions such as regular expressions and other advanced selection logic that can be seamlessly combined using these powerful selectors.

### DescendantCombinator

The **Descendant Combinator** is one of the simplest and most frequently used combinators in CSS. It selects elements that match a second selector only if they have an ancestor that matches the first selector. In CSS, this relationship is represented by a single space `" "` between two selectors. For example, the following CSS:

```css
div.book p.price
```

matches all `<p>` tags with the class `price` that are descendants of `<div>` tags with the class `book`. For more details on CSS combinators, refer to the [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/Descendant_combinator).

#### Using DescendantCombinator in `soupsavvy`

In `soupsavvy`, this logic is encapsulated in the `DescendantCombinator` class, which functions similarly to its CSS counterpart. The `DescendantCombinator` accepts two or more selectors and returns `Tag` objects that satisfy the descendant relationship between them.

Here's an example equivalent to the CSS above:

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, DescendantCombinator, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <span class="price_section">
                <p class="price">Price: $20</p>
            </span>
        </div>
    """,
    features="lxml",
)

book_selector = TagSelector(
    "div", attributes=[AttributeSelector("class", value="book")]
)
price_selector = TagSelector(
    "p", attributes=[AttributeSelector("class", value="price")]
)
selector = DescendantCombinator(book_selector, price_selector)
selector.find(soup)

As mentioned in previously, `soupsavvy` provides alternative way of creating some composite selectors by using operators. More concise way to create `DescendantCombinator` is `>>` operator, which acts as syntactic sugar:

```python
DescendantCombinator(left, right) == left >> right
```

Where left selector matches ancestor and right selector matches descendant.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, TagSelector, DescendantCombinator

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <span class="price_section">
                <p class="price">Price: $20</p>
            </span>
        </div>
    """,
    features="lxml",
)

book_selector = TagSelector(
    "div", attributes=[AttributeSelector("class", value="book")]
)
price_selector = TagSelector(
    "p", attributes=[AttributeSelector("class", value="price")]
)
selector = book_selector >> price_selector
selector.find(soup)

The order of selectors in a `DescendantCombinator` is significant. `left >> right` is not the same as `right >> left`.

In [ ]:
from soupsavvy import AttributeSelector, DescendantCombinator, TagSelector

book_selector = TagSelector(
    "div", attributes=[AttributeSelector("class", value="book")]
)
price_selector = TagSelector(
    "p", attributes=[AttributeSelector("class", value="price")]
)
print(
    "left >> right == DescendantCombinator(left, right):",
    book_selector >> price_selector
    == DescendantCombinator(book_selector, price_selector),
)
print(
    "left >> right == right >> left:",
    price_selector >> book_selector == book_selector >> price_selector,
)

#### Handling Multiple Selectors

`DescendantCombinator` allows you to chain together any number of selectors as positional arguments. When more than two selectors are provided, they are chained them in the order they appear, creating a more complex selection logic. This is similar to chaining selectors in CSS.

For instance, the following CSS:

```css
div#available div.book .price
```

matches all elements with the class `price` that are descendants of a `<div>` with the class `book`, which in turn is a descendant of a `<div>` with the ID `available`.

In `soupsavvy`, you can achieve this with `DescendantCombinator` as shown below:

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, IdSelector, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <p class="price">Price: $10</p>
        </div>
        <div id="available">
            <div class="book">
                <span class="title">Animal Farm</span>
                <span class="price_section">
                    <p class="price">Price: $20</p>
                </span>
            </div>
        </div>
    """,
    features="lxml",
)

available_selector = TagSelector("div", attributes=[IdSelector("available")])
book_selector = TagSelector("div", attributes=[ClassSelector("book")])
price_selector = ClassSelector("price")
selector = available_selector >> book_selector >> price_selector
selector.find(soup)

#### Not Recursive search

Setting `recursive` parameter to `False` will return element only if element matched by first selector is a child of searched element. This logic is in place for all `soupsavvy` combinators.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, TagSelector, DescendantCombinator

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <span class="not_child_book">
            <div class="book">
                <span class="title">Animal Farm</span>
                <span class="price_section">
                    <p class="price">Price: $50</p>
                </span>
            </div>
        </span>
        <div class="book">
            <span class="title">Animal Farm</span>
            <span class="price_section">
                <p class="price">Price: $20</p>
            </span>
        </div>
    """,
    features="html.parser",
)

book_selector = TagSelector(
    "div", attributes=[AttributeSelector("class", value="book")]
)
price_selector = TagSelector(
    "p", attributes=[AttributeSelector("class", value="price")]
)
selector = book_selector >> price_selector
selector.find(soup, recursive=False)

In this case `Price: $50` is not selected, as it's ancestor `<div class="book">` that matched first selector is not a direct parent of searched element.

### ChildCombinator

`ChildCombinator` mirrors the behavior of the CSS child combinator. In CSS, a child combinator (`>` symbol) selects elements that are direct children of a specified parent element. This is a stricter relationship than the descendant combinator, as it only matches elements that are immediate children of a given element, not just any descendant.

For example, in CSS:

```css
div.book > p.price
```

This selector matches all `<p>` elements with the class `price` that are direct children of `<div>` elements with the class `book`. For more details refer to [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/Child_combinator).

In `soupsavvy`, this logic is implemented with the `ChildCombinator` class. It accepts two or more selectors as positional arguments and returns elements that match the specified child-parent relationships.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, ChildCombinator, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <span class="discount">
                <h2>Discounted</h2>
                <p class="price">Price: $15</p>
            </span>
            <p class="price">Price: $20</p>
        </div>
    """,
    features="lxml",
)

book_selector = TagSelector(
    "div", attributes=[AttributeSelector("class", value="book")]
)
price_selector = TagSelector(
    "p", attributes=[AttributeSelector("class", value="price")]
)
selector = ChildCombinator(book_selector, price_selector)
selector.find(soup)

More concise way to create `ChildCombinator` is (no surprise) `>` operator, which is consistent with CSS syntax:

```python
ChildCombinator(left, right) == left > right
```

Where left selector matches parent and right selector matches child.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <span class="discount">
                <h2>Discounted</h2>
                <p class="price">Price: $15</p>
            </span>
            <p class="price">Price: $20</p>
        </div>
    """,
    features="lxml",
)

book_selector = TagSelector("div", attributes=[ClassSelector("book")])
price_selector = TagSelector("p", attributes=[ClassSelector("price")])
selector = book_selector > price_selector
selector.find(soup)

### Combining Combinators

Just like any other `soupsavvy` selector, combinators can be used as inputs to other higher-order selectors. For instance, you can define a combination of parent-child and descendant relationships within a single selector. In CSS, this would be represented as:

```css
div#available > div.book p.price
```

In `soupsavvy`, this logic can be replicated by using both the `ChildCombinator` and the `DescendantCombinator` together. This combination allows you to specify that a `<div>` with the ID `available` should contain a direct child `<div>` with the class `book`, which in turn should contain a descendant `<p>` element with the class `price`.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, IdSelector, TagSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <p class="price">Price: $10</p>
        </div>
        <div id="available">
            <div class="book">
                <span class="title">Animal Farm</span>
                <span class="discount">
                    <h2>Discounted</h2>
                    <p class="price">Price: $15</p>
                </span>
                <p class="price">Price: $20</p>
            </div>
        </div>
    """,
    features="lxml",
)

available_selector = TagSelector("div", attributes=[IdSelector("available")])
book_selector = TagSelector("div", attributes=[ClassSelector("book")])
price_selector = ClassSelector("price")
selector = available_selector > book_selector >> price_selector
selector.find(soup)

When combining selectors with operators in `soupsavvy`, it's important to understand that some operators have higher precedence than others, which can affect the order in which expressions are evaluated.

For example, the `right shift` operator (`>>`) has higher precedence than the `greater than` operator (`>`). This means that in an expression like:

```python
left > middle >> right
```

The `>>` operation is performed first, resulting in:

```python
ChildCombinator(left, DescendantCombinator(middle, right))
```

In contrast, if you add parentheses to explicitly define the order of operations:

```python
(left > middle) >> right
```

This expression would be interpreted as:

```python
DescendantCombinator(ChildCombinator(left, middle), right)
```

In provided example, the order of selectors isn't significant because both the `ChildCombinator` and `DescendantCombinator` are commutative — meaning the order in which you apply them doesn't affect the final result. However, this may not always be the case, so to avoid ambiguity and ensure expected behavior, it's always safer to include parentheses when combining operators.

In [ ]:
from soupsavvy import ClassSelector, DescendantCombinator, IdSelector, TagSelector, ChildCombinator

available_selector = TagSelector("div", attributes=[IdSelector("available")])
book_selector = TagSelector("div", attributes=[ClassSelector("book")])
price_selector = ClassSelector("price")

print(
    DescendantCombinator(
        ChildCombinator(available_selector, book_selector), price_selector
    )
    == (available_selector > book_selector) >> price_selector
)

### NextSiblingCombinator

`NextSiblingCombinator` replicates the behavior of the CSS adjacent sibling combinator. In CSS, the next (aka adjacent) sibling combinator (denoted by the `+` symbol) selects an element that directly follows a specified sibling element.

For example, in CSS:

```css
div.book + p.price
```

This selector matches all `<p>` elements with the class `price` that are the immediate next siblings of `<div>` elements with the class `book`. For more details, refer to [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/Next-sibling_combinator).

In `soupsavvy`, the `NextSiblingCombinator` class implements this logic. It takes two or more selectors as positional arguments and returns elements that match the specified preceding-adjacent sibling relationships.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, NextSiblingCombinator, PatternSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <span class="discount">
            <h2>Discounted</h2>
            <p class="price">Price: $15</p>
            <h2>Highest Price</h2>
            <p class="price">Price: $20</p>
        </span>
        <p class="price">Price: $20</p>
    """,
    features="lxml",
)

discount_selector = PatternSelector("Discounted")
price_selector = ClassSelector("price")
selector = NextSiblingCombinator(discount_selector, price_selector)
selector.find(soup)

More concise way to create `NextSiblingCombinator` is `+` operator, which is consistent with CSS syntax:

```python
NextSiblingCombinator(left, right) == left + right
```

Where left selector matches preceding sibling and right selector matches next sibling.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, PatternSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <span class="discount">
            <h2>Discounted</h2>
            <p class="price">Price: $15</p>
            <h2>Highest Price</h2>
            <p class="price">Price: $20</p>
        </span>
        <p class="price">Price: $30</p>
    """,
    features="lxml",
)

discount_selector = PatternSelector("Discounted")
price_selector = ClassSelector("price")
selector = discount_selector + price_selector
selector.find(soup)

In case of having multiple selectors, they are chained in the order they appear, first selector matches element and each subsequent selector matches next sibling of previously matched element.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, PatternSelector

soup = BeautifulSoup(
    """
        <h2>Discounted</h2>
        <span>Breaking relationship :(</span>
        <p class="price">Price: $15</p>
        <span class="title">Animal Farm 1</span>
        
        <h2>Discounted</h2>
        <p class="price">Price: $15</p>
        <span>Breaking relationship :(</span>
        <span class="title">Animal Farm 2</span>
        
        <h2>Discounted</h2>
        <p class="price">Price: $15</p>
        <span class="title">Animal Farm 3</span>
    """,
    features="lxml",
)

discount_selector = PatternSelector("Discounted")
price_selector = ClassSelector("price")
title_selector = ClassSelector("title")
selector = discount_selector + price_selector + title_selector
selector.find(soup)

### SubsequentSiblingCombinator

`SubsequentSiblingCombinator` emulates the behavior of the CSS subsequent sibling combinator. In CSS, the subsequent sibling combinator (denoted by the `~` symbol) selects elements that follow the element matched by the first selector and share the same parent.

For example, in CSS:

```css
div.book ~ p.price
```

This selector matches all `<p>` elements with the class `price` that are siblings of `<div>` elements with the class `book`, and appear after it in document. For more details, refer to [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/Subsequent-sibling_combinator).

In `soupsavvy`, the `SubsequentSiblingCombinator` class provides similar functionality. It accepts two or more selectors as positional arguments and returns elements that match the specified preceding-subsequent sibling relationships. For more information on search logic when multiple selectors are provided, refer to the `NextSiblingCombinator` section.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import (
    ClassSelector,
    SubsequentSiblingCombinator,
    TagSelector,
)

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <span class="discount">
            <p class="price">Price: $25</p>
            <h2>Discounted</h2>
            <span>Bargain!!!</span>
            <p class="price">Price: $15</p>
        </span>
        <p class="price">Price: $20</p>
    """,
    features="lxml",
)

discount_selector = TagSelector("h2")
price_selector = ClassSelector("price")
selector = SubsequentSiblingCombinator(discount_selector, price_selector)
selector.find(soup)

More concise way to create `SubsequentSiblingCombinator` is `~` operator:

```python
SubsequentSiblingCombinator(left, right) == left ~ right
```

Where left selector matches preceding sibling and right selector matches subsequent sibling.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, TagSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <span class="discount">
            <p class="price">Price: $25</p>
            <h2>Discounted</h2>
            <span>Bargain!!!</span>
            <p class="price">Price: $15</p>
        </span>
        <p class="price">Price: $20</p>
    """,
    features="lxml",
)

discount_selector = TagSelector("h2")
price_selector = ClassSelector("price")
selector = discount_selector * price_selector
selector.find(soup)

## Logical Selectors

Another category of higher-order selectors in `soupsavvy` is combination with logical operators. These selectors allow you to combine multiple selectors using logical operators such as `AND`, `OR`, `XOR` and `NOT`, enabling you to create more complex search conditions.

## AndSelector

```css
p.price#best
```

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import (
    AndSelector,
    ClassSelector,
    IdSelector,
    PatternSelector,
    TagSelector,
)

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="price">Price: $30</p>
        <p class="price">Price: $20</p>
        <p class="price" id="best">Price: €12</p>
        <p class="price" id="best">Price: $15</p>
    """,
    features="lxml",
)

price_selector = TagSelector(
    "p", attributes=[ClassSelector("price"), IdSelector("best")]
)
dollars_selector = PatternSelector(re.compile(r"\$\d+"))
selector = AndSelector(price_selector, dollars_selector)
selector.find(soup)

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import (
    ClassSelector,
    IdSelector,
    PatternSelector,
    TagSelector,
)

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="price">Price: $30</p>
        <p class="price">Price: $20</p>
        <p class="price" id="best">Price: €12</p>
        <p class="price" id="best">Price: $15</p>
    """,
    features="lxml",
)

selector = (
    TagSelector("p")
    & ClassSelector("price")
    & IdSelector("best")
    & PatternSelector(re.compile(r"\$\d+"))
)
selector.find(soup)

## OrSelector (SelectorList)

```css
h1, h1 { color: red;}
:is(h1, h2) { color: red; }
```


In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import (
    ClassSelector,
    OrSelector,
    PatternSelector,
)

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <span class="discount">
            <h2>Discounted</h2>
            <p class="price">Price: $25</p>
            <p class="price"><s>Price: $30</s></p>
            <p class="price"><s>Price: $35</s></p>
        </span>
        <p class="title">Brave New World</p>
        <p class="price">Price: $15</p>
    """,
    features="lxml",
)

discount_selector = ClassSelector("discount") > PatternSelector(
    "Discounted"
) + ClassSelector("price")
standard_price_selector = ClassSelector("title") + PatternSelector(re.compile(r"\$\d+"))
selector = OrSelector(discount_selector, standard_price_selector)
selector.find_all(soup)

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import (
    ClassSelector,
    PatternSelector,
)

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <span class="discount">
            <h2>Discounted</h2>
            <p class="price">Price: $25</p>
            <p class="price"><s>Price: $30</s></p>
            <p class="price"><s>Price: $35</s></p>
        </span>
        <p class="title">Brave New World</p>
        <p class="price">Price: $15</p>
    """,
    features="lxml",
)

discount_selector = ClassSelector("discount") > PatternSelector(
    "Discounted"
) + ClassSelector("price")
standard_price_selector = ClassSelector("title") + PatternSelector(re.compile(r"\$\d+"))
selector = discount_selector | standard_price_selector
selector.find_all(soup)

In [ ]:
from soupsavvy import ClassSelector, OrSelector, SelectorList

discount_selector = ClassSelector("discount")
price_selector = ClassSelector("price")

print(
    SelectorList(discount_selector, price_selector)
    == discount_selector | price_selector
    == OrSelector(discount_selector, price_selector)
)

In [ ]:
from soupsavvy import ClassSelector, SelectorList

discount_selector = ClassSelector("discount")
price_selector = ClassSelector("price")

print(
    SelectorList(discount_selector, price_selector)
    == SelectorList(price_selector, discount_selector)
)

## XOR Selector

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, XORSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <a href="/shop">Buy!</a>
        <p class="price" href="/shop">New price! Discount!</p>
        <p class="price">Price: $30</p>
    """,
    features="lxml",
)

price_selector = AttributeSelector("class", value="price")
shop_selector = AttributeSelector("href", value="/shop")
selector = XORSelector(price_selector, shop_selector)
selector.find_all(soup)

## NotSelector

```css	
p:not(.price)
```

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, NotSelector

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = NotSelector(discount_selector)
selector.find(soup)

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import (
    ClassSelector,
    NotSelector,
    PatternSelector,
)

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
dollars_selector = PatternSelector(re.compile(r"\$\d+"))
selector = NotSelector(discount_selector, dollars_selector)
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = ~discount_selector
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = ~NotSelector(discount_selector)
print(selector)
selector.find(soup)

## HasSelector

```css
div:has(p)
```

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Brave New World</span>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Animal Farm</span>
        <span class="discount">
            <p class="price">Price: $15</p>
        </span>
        <p class="price">Price: $20</p>
    </div>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = HasSelector(discount_selector)
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector, PatternSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Brave New World</span>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Animal Farm</span>
        <span class="discount">
            <p class="price">Price: $15</p>
        </span>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span>Bestseller</span>
        <span class="title">Frankenstein</span>
        <p class="price">Price: $30</p>
    </div>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
bestseller_selector = PatternSelector("Bestseller")
selector = HasSelector(discount_selector, bestseller_selector)
selector.find_all(soup)

## Relative Selectors

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy.selectors.relative import RelativeChild
from soupsavvy import AttributeSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $30</p>
    """,
    features="html.parser",
)

selector = AttributeSelector("class", value="price")
relative = RelativeChild(selector)
relative.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, Anchor

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $30</p>
    """,
    features="html.parser",
)

selector = AttributeSelector("class", value="price")
relative = Anchor > selector
print(type(relative))
relative.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector
from soupsavvy.selectors.relative import RelativeChild

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $30</p>
        <div>
            <p class="price">Price: $40</p>
        </div>
    """,
    features="html.parser",
)

selector = AttributeSelector("class", value="price")
relative = RelativeChild(selector)

print(relative.find_all(soup, recursive=False))
print(relative.find_all(soup, recursive=True))

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy.selectors.relative import RelativeNextSibling
from soupsavvy import AttributeSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <div class="section">Book 1</div>
        <p class="price">Price: $30</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $10</p>
    """,
    features="lxml",
)

selector = AttributeSelector("class", value="price")
relative = RelativeNextSibling(selector)
relative.find_all(soup.div)  # type: ignore

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, Anchor

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $30</p>
        <div>
            <p class="price">Price: $40</p>
            <a href="/shop">Buy Now</a>
        </div>
    """,
    features="html.parser",
)

price_selector = AttributeSelector("class", value="price")
shop_selector = AttributeSelector("href", value="/shop")
and_selector = (Anchor > price_selector) | shop_selector
and_selector.find_all(soup, recursive=True)

## RelativeSelectors with HasSelector

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy.selectors.relative import RelativeNextSibling
from soupsavvy import AttributeSelector, HasSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <div class="section">Book 1</div>
        <p class="price">Price: $30</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $10</p>
    """,
    features="lxml",
)

selector = AttributeSelector("class", value="price")
relative = RelativeNextSibling(selector)
has_selector = HasSelector(relative)
has_selector.find_all(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, HasSelector, Anchor, TagSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <div class="section">Book 1</div>
        <p class="price">Price: $30</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $10</p>
    """,
    features="lxml",
)

relative = Anchor + AttributeSelector("class", value="price")
has_selector = HasSelector(relative) & TagSelector("div")
has_selector.find_all(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, HasSelector, Anchor, PatternSelector

soup = BeautifulSoup(
    """
        <div class="newspaper">
            <p class="title">New York Times</p>
            <h1>Only 5$!</h1>
        </div>
        <p>Read more</p>
        <div class="book">
            <span>Animal Farm</span>
            <p class="price">Price: $30</p>
        </div>
    """,
    features="lxml",
)

relative_newspaper = Anchor + PatternSelector("Read more")
relative_book = Anchor > AttributeSelector("class", value="price")
has_selector = HasSelector(relative_newspaper, relative_book)
has_selector.find_all(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import HasSelector, Anchor, PatternSelector

soup = BeautifulSoup(
    """
        <div class="countdown">4</div>
        <span>3</span>
        <p id="fgai23">2</p>
        <a href="https://example.com">1</a>
        <span>Stop</span>
        <p>Go</p>
    """,
    features="lxml",
)

relative_countdown = Anchor * PatternSelector("Stop")
has_selector = HasSelector(relative_countdown)
has_selector.find_all(soup)

## NthOfSelector

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector
from soupsavvy.selectors.nth import NthOfSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <span>Bestseller</span>
        <p class="price">Price: €15</p>
        <p class="price">Price: €25</p>
        <p class="price">Price: €17</p>
    """,
    features="lxml",
)

price_selector = ClassSelector("price")
selector = NthOfSelector(price_selector, nth="2n")
selector.find_all(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector
from soupsavvy.selectors.nth import NthLastOfSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <span>Bestseller</span>
        <p class="price">Price: €15</p>
        <p class="price">Price: €25</p>
        <p class="price">Price: €17</p>
    """,
    features="lxml",
)

price_selector = ClassSelector("price")
selector = NthLastOfSelector(price_selector, nth="odd")
selector.find_all(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector
from soupsavvy.selectors.nth import OnlyOfSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Animal Farm</span>
        <p class="price">Price: $15</p>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Frankenstein</span>
        <p class="price">Price: $30</p>
    </div>
    """,
    features="html.parser",
)
price_selector = ClassSelector("price")
selector = HasSelector(OnlyOfSelector(price_selector))
selector.find(soup)

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector
from soupsavvy.selectors.nth import OnlyOfSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Frankenstein</span>
        <p class="price">Price: $30</p>
    </div>
    <div class="book">
        <span class="title">Animal Farm</span>
        <p class="price">Price: $15</p>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Brave New world</span>
        <p class="price">Price: $25</p>
        <p class="price">Price: $20</p>
    </div>
    """,
    features="html.parser",
)
price_selector = ClassSelector("price")
selector = ~HasSelector(OnlyOfSelector(price_selector)) > ClassSelector("title")
selector.find_all(soup, recursive=False)

## Overview